In [ ]:
import torch
import non_local_boxes
import matplotlib.pyplot as plt

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Implementation of `rho(P)`

In [ ]:
# def BG15_form(P): # P is a 4x4 box, in the usual form
#     P = non_local_boxes.utils.matrix_to_tensor(P)
#     Q = torch.zeros(4,4)
#     for i in range(4):
#         a, x = i//2, i%2
#         for j in range(4):
#             b, y = j//2, j%2
#             Q[i,j] = P[a, b, x, y]
#     return Q

def BG15_form(P): # P is a 4x4 box, in the usual form
    P = non_local_boxes.utils.matrix_to_tensor(P)
    Q = torch.zeros(4,4)
    for i in range(4):
        a, x = i//2, i%2
        for j in range(2):
            # b, y = j, x
            Q[i, 2*j + x] = P[a, j, x, x]
    return Q

In [ ]:
def marginals(P):  # P is a 4x4 matrix, in the BS15 form
    e = torch.ones(4)
    pX = torch.tensordot(P, e, dims=1)
    pY = torch.tensordot(torch.t(P), e, dims=1)
    return pX, pY

def marginals_diag(P):  # P is a 4x4 matrix, in the BS15 form
    pX, pY = marginals(P)
    P_X, P_Y = torch.diag_embed(pX), torch.diag_embed(pY)
    return P_X, P_Y

In [ ]:
def P_tilda(P):  # P is a 4x4 matrix, in the usual form
    P = BG15_form(P)
    pX, pY = marginals(P)
    A, B = torch.diag_embed(torch.pow(pX, -0.5)), torch.diag_embed(torch.pow(pY, -0.5))
    return torch.tensordot(torch.tensordot(A, P, dims=1), B, dims=1)

In [ ]:
def rho(P):   # P is a 4x4 matrix, in the usual form
    sing_val = torch.linalg.svdvals(P_tilda(P))
    return round(float(sing_val[2]), 6)

# Compute $\rho$ of somes boxes

In [ ]:
PR = non_local_boxes.utils.PR
SR = non_local_boxes.utils.SR
I = non_local_boxes.utils.I

In [ ]:
print(rho(PR), rho(SR), rho(I))

In [ ]:
def draw_triangle(Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size): # Box is a 4x4 tensor
    X = torch.tensor([[0, 10], [10,0], [0, 0]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)

    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            p = rho(alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3)
            color_point = ((1-p)*0., 0.*(1-p)+0.8*p, 0.*(1-p)+0.8*p)
            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)

    plt.text(X[0,0]+0.3, X[0,1], Box1_name, horizontalalignment='center')
    plt.text(X[1,0], X[1,1]+0.3, Box2_name, verticalalignment='center')
    plt.text(X[2,0], X[2,1]-0.3, Box3_name, verticalalignment='center')
    plt.show()

In [ ]:
draw_triangle(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 40
)